Usaremos una RNN del tipos LSTM para predecir los valores de bolsa de una compañía. Haremos servir los valores de la bolsa NASDAQ (National Association of Securities Dealer Automated Quotation), que es la más grande de Estados Unidos.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import pandas as pd

In [ ]:
# Leemos el conjunto de datos, que descargaremos utilizando la biblioteca yfinance
import yfinance as yf

# Descargamos los datos de Google de NASDAQ
# Leeremos los datos desde el año 2012 hasta enero de 2024
# Empresas posibles para consultar --> Google: GOOGL, Amazon: AMZN, Apple: AAPL, Netflix: NFLX
# Podéis consultar el acrónimo de otras empresas en https://es.wikipedia.org/wiki/NASDAQ-100
empresa = 'GOOGL'
datos = yf.download(empresa, start='2012-01-01', end='2024-01-01')

# Muestra las primeras filas de los datos
print(datos.head())


In [ ]:
# Nos quedamos con los datos de cierre y los visualizamos.
datos = datos['Close']
plt.plot(datos);
print(datos.head())


In [ ]:
# Normalizamos los datos
escalador = MinMaxScaler()
array_de_datos = np.array(datos)
datos_escalados = escalador.fit_transform(array_de_datos.reshape(-1, 1))
print("Los datos más antiguos:")
print(datos_escalados[0:5])
print("Los datos más recientes:")
print(datos_escalados[-5::])


In [ ]:
# Dividimos las muestras en entrenamiento y test
tamaño_entrenamiento = int(len(datos_escalados) * 0.8)
tamaño_prueba = len(datos_escalados) - tamaño_entrenamiento
datos_entrenamiento = datos_escalados[0:tamaño_entrenamiento, :]
datos_prueba = datos_escalados[tamaño_entrenamiento:len(datos_escalados), :1]


In [ ]:
# Creamos pequeñas series temporales para entrenar y testear
def crear_series_temporales(datos, longitud_serie):
    X, Y = [], []
    for i in range(0, len(datos) - longitud_serie - 1):
        serie_X = datos[i:(i + longitud_serie)]  # Secuencia de datos hasta el momento actual
        serie_Y = datos[i + longitud_serie]  # Valor siguiente a predecir
        X.append(serie_X)
        Y.append(serie_Y)
    return np.array(X), np.array(Y)


longitud_serie = 10  # Longitud de la ventana de la serie temporal
X_train, y_train = crear_series_temporales(datos_entrenamiento, longitud_serie)
X_test, y_test = crear_series_temporales(datos_prueba, longitud_serie)
print("Los datos de entrenamiento tienen la forma {}".format(X_train.shape))
print("Los valores a predecir de entrenamiento (y) tienen la forma {}".format(y_train.shape))
print("Los datos de prueba tienen la forma {}".format(X_test.shape))
print("Los valores a predecir de prueba (y) tienen la forma {}".format(y_test.shape))


In [ ]:
# Eliminamos la última dimensión:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1])
y_train = y_train.reshape(y_train.shape[0])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1])
y_test = y_test.reshape(y_test.shape[0])


In [ ]:
# Definimos y compilamos el modelo LSTM
modelo = Sequential()
modelo.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], 1)))
modelo.add(Dense(1))
modelo.compile(optimizer='adam', loss='mse')


In [ ]:
# Entrenamos el modelo
modelo.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1)

# Evaluamos el modelo
pérdida_entrenamiento = modelo.evaluate(X_train, y_train, verbose=0)
pérdida_prueba = modelo.evaluate(X_test, y_test, verbose=0)
print('Pérdida de entrenamiento:', pérdida_entrenamiento)
print('Pérdida de prueba:', pérdida_prueba)


In [ ]:
# Usamos los datos para predecir
predicciones_entrenamiento = modelo.predict(X_train)
predicciones_prueba = modelo.predict(X_test)
# Desnormalizamos las predicciones
predicciones_entrenamiento = escalador.inverse_transform(predicciones_entrenamiento)
predicciones_prueba = escalador.inverse_transform(predicciones_prueba)
# Creamos nuevos DataFrames con el índice original
predicciones_entrenamiento = pd.DataFrame(predicciones_entrenamiento, index=datos.index[:len(predicciones_entrenamiento)])
predicciones_prueba = pd.DataFrame(predicciones_prueba, index=datos.index[len(datos)-len(predicciones_prueba):])


In [ ]:
# Gráfica de todas las predicciones
plt.plot(datos, label='Datos reales')
plt.plot(predicciones_entrenamiento, label='Predicciones entrenamiento')
plt.plot(predicciones_prueba, label='Predicciones prueba')
plt.legend()
plt.show()
